

# Examen Técnico GenAI Tech V1.0



## Índice

- [Descripción del Examen](#descripción-del-examen)
- [Restricción Importante](#restricción-importante)
- [Pasos del Examen](#pasos-del-examen)
- [Parte 1: Configuración y Extracción de Datos](#parte-1-configuración-y-extracción-de-datos)
- [Parte 2: Procesamiento y Categorización de Transacciones](#parte-2-procesamiento-y-categorización-de-transacciones)
- [Parte 3: Búsqueda y Almacenamiento de Logos](#parte-3-búsqueda-y-almacenamiento-de-logos)
- [Parte 4: Documentación y Pruebas](#parte-4-documentación-y-pruebas)





## Descripción del Examen



El propósito de este examen técnico es desarrollar un sistema en Python capaz de procesar y categorizar veinticinco transacciones bancarias ficticias almacenadas en una base de datos. Los candidatos deberán limpiar los nombres de las transacciones para obtener un "cleaned_transaction" que represente el nombre del comercio, asignar una categoría adecuada a cada transacción y buscar y guardar el logotipo del comercio utilizando una herramienta de LangChain.




## Restricción Importante


Uso de Prompts: Queda estrictamente prohibido utilizar el nombre de los comercios directamente en cualquier prompt de API, incluyendo, pero no limitándose a, servicios de búsqueda de imágenes o generación de texto. Esto es para asegurar que la implementación cumpla con las mejores prácticas de seguridad y privacidad de datos.
Requisitos Técnicos
Plataforma: Jupyter Notebook o Google Colab.
Lenguajes de Programación: Python.
Librerías Sugeridas: Pandas, SQLAlchemy, Requests.
APIs y Servicios Externos: LangChain para búsqueda de logos, OpenAI para procesamiento de lenguaje natural.


## Pasos del Examen



- Parte 1: Configuración y Extracción de Datos
Configuración del Entorno: Instalar las librerías necesarias.
Conexión a la Base de Datos: Configurar la conexión a la base de datos con SQLAlchemy.
Carga de Datos Ficticios: Cargar veinticinco transacciones inventadas en la base de datos.

- Parte 2: Procesamiento y Categorización de Transacciones
Limpieza de Datos: Limpiar los datos para extraer nombres claros de los comercios ("cleaned_transaction").
Categorización de Transacciones: Aplicar técnicas de NLP para asignar categorías a las transacciones.
- Parte 3: Búsqueda y Almacenamiento de Logos
Búsqueda de Logos: Utilizar LangChain para buscar logos de comercios identificados.
Almacenamiento de Logos: Guardar los logos en la base de datos.
-Parte 4: Documentación y Pruebas
Documentación del Proceso: Documentar detalladamente todas las etapas y el código.
Pruebas de Funcionalidad: Realizar pruebas para verificar la correcta funcionalidad del sistema.

Datos
Transacciones:
- DEP NOMINA BBVA 12072024
- CAJERO BBVA BANCOMER ZONA ROSA 003456
- TRANSFER SPEI RECIBIDO BANORTE0025
- PAGO TARJETA VISA AMAZON MKTPLACE PMTS
- STRIPE*UBEREATSMX00567
- RETIRO CAJERO AUTOMATICO HSBC AV. REFORMA 0998
- CARGA SUBURBIA ONLINE 0645 REF 193485
- SPOTIFY P1401201C5 REF SPOTIFY TECHNOLOGY
- OXXO VENTAS TIENDA 3500 MX 063587
- TAR RETIRO SUC 1205 BANAMEX CDMX
- PAPELERIA TONY VTA DIRECTA ONLINE MX 014725
- PAGO SERVICIO CFE CDMX 25072024
- TRANSFER INTERBANCARIA RECIBIDA CLABE 032580009600975835
- *PAYPAL EBAY 4423
- NETFLIX.COM 223 MX EN LINEA
- TIENDA3B REFORMA 2453 COMPRA CHEQUE
- RENTA AUTOMATICA DEP DEPTO 302 A BANCOMER
- COMPRA APPLE STORE ONLINE MX REF 593465
- IZZI TELECOM PAGO SERV 31072024
- CARGO AUTOMATICO GYM ENERGY FITNESS REF MX 0426

- Evaluación
La evaluación se centrará en la precisión en la limpieza de datos y categorización, la implementación correcta de la búsqueda de logos con LangChain, y la calidad de la documentación y pruebas realizadas. La adherencia a la restricción establecida será un factor crucial en la evaluación final.



## Parte 1: Configuración y Extracción de Datos



In [1]:

!pip install pandas sqlalchemy requests langchain openai python-dotenv langchain-community tavily-python   langchain-openai langchainhub spacy


se configura el entorno con las dependencias que deben de ser instaladas

In [2]:
from dotenv import load_dotenv
from sqlalchemy import create_engine, Column, String, Integer, Float, MetaData, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import re
import openai 
import os
from openai import OpenAI
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents import initialize_agent, AgentType
from langchain_community.chat_models import ChatOpenAI
from langchain.tools.tavily_search import TavilySearchResults
import pandas as pd
import spacy

Se importan las dependencias que se utilizarán en el ejercicio en un solo sitio para mantenerlas para siguientes consultas

In [3]:
engine = create_engine("sqlite:///transacciones.db", echo=True)
Base = declarative_base()

class Transaccion(Base):
    __tablename__ = "transacciones"
    id = Column(Integer, primary_key=True)
    descripcion = Column(String(100))
    cleaned_transaction = Column(String(30))
    categoria = Column(String(100))
    logo = Column(String())

Posteriormente creamos la base de datos con nombre "transacciones" de forma declarativa , se elige esta forma por que mantiene la intuición de python y para que de esta forma pueda ser más accesible globalmente en el entorno, se crean las columnas descripcion, cleaned_transaction, categoria y logo de tipo String, mientras id se elige ser tipo Integer.

In [4]:
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

2024-07-24 07:39:01,414 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-24 07:39:01,414 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("transacciones")
2024-07-24 07:39:01,415 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-24 07:39:01,416 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("transacciones")
2024-07-24 07:39:01,417 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-24 07:39:01,418 INFO sqlalchemy.engine.Engine 
CREATE TABLE transacciones (
	id INTEGER NOT NULL, 
	descripcion VARCHAR(100), 
	cleaned_transaction VARCHAR(30), 
	categoria VARCHAR(100), 
	logo VARCHAR, 
	PRIMARY KEY (id)
)


2024-07-24 07:39:01,418 INFO sqlalchemy.engine.Engine [no key 0.00085s] ()
2024-07-24 07:39:01,436 INFO sqlalchemy.engine.Engine COMMIT


Iniciamos la base de datos declarativa con el subfijo Base y creamos una sesión para conectarnos con la base de datos recien creada

In [63]:
transacciones = [
    "DEP NOMINA BBVA 12072024",
    "CAJERO BBVA BANCOMER ZONA ROSA 003456",
    "TRANSFER SPEI RECIBIDO BANORTE0025",
    "PAGO TARJETA VISA AMAZON MKTPLACE PMTS",
    "STRIPE*UBEREATSMX00567",
    "RETIRO CAJERO AUTOMATICO HSBC AV. REFORMA 0998",
    "CARGA SUBURBIA ONLINE 0645 REF 193485",
    "SPOTIFY P1401201C5 REF SPOTIFY TECHNOLOGY",
    "OXXO VENTAS TIENDA 3500 MX 063587",
    "TAR RETIRO SUC 1205 BANAMEX CDMX",
    "PAPELERIA TONY VTA DIRECTA ONLINE MX 014725",
    "PAGO SERVICIO CFE CDMX 25072024",
    "TRANSFER INTERBANCARIA RECIBIDA CLABE 032580009600975835",
    "*PAYPAL EBAY 4423",
    "NETFLIX.COM 223 MX EN LINEA",
    "TIENDA3B REFORMA 2453 COMPRA CHEQUE",
    "RENTA AUTOMATICA DEP DEPTO 302 A BANCOMER",
    "COMPRA APPLE STORE ONLINE MX REF 593465",
    "IZZI TELECOM PAGO SERV 31072024",
    "CARGO AUTOMATICO GYM ENERGY FITNESS REF MX 0426",
]

Declaramos los datos en una lista de python para integrarlos a la base de datos.

In [6]:
for descripcion in transacciones:
    nueva_transaccion = Transaccion(descripcion=descripcion)
    session.add(nueva_transaccion)
session.commit()

2024-07-24 07:39:01,547 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-24 07:39:01,550 INFO sqlalchemy.engine.Engine INSERT INTO transacciones (descripcion, cleaned_transaction, categoria, logo) VALUES (?, ?, ?, ?)
2024-07-24 07:39:01,551 INFO sqlalchemy.engine.Engine [generated in 0.00081s] ('DEP NOMINA BBVA 12072024', None, None, None)
2024-07-24 07:39:01,555 INFO sqlalchemy.engine.Engine INSERT INTO transacciones (descripcion, cleaned_transaction, categoria, logo) VALUES (?, ?, ?, ?)
2024-07-24 07:39:01,556 INFO sqlalchemy.engine.Engine [cached since 0.005911s ago] ('CAJERO BBVA BANCOMER ZONA ROSA 003456', None, None, None)
2024-07-24 07:39:01,557 INFO sqlalchemy.engine.Engine INSERT INTO transacciones (descripcion, cleaned_transaction, categoria, logo) VALUES (?, ?, ?, ?)
2024-07-24 07:39:01,557 INFO sqlalchemy.engine.Engine [cached since 0.007424s ago] ('TRANSFER SPEI RECIBIDO BANORTE0025', None, None, None)
2024-07-24 07:39:01,557 INFO sqlalchemy.engine.Engine INSERT INTO

Realizamos un **bucle for** para iterar cada elemento de la lista transacciones en la base de datos que invocamos como objeto, para rellenar el paramétro **descripción** en una variable llamada **nueva transacción** con el fin de realizar un update en la base de datos.



## Parte 2: Procesamiento y Categorización de Transacciones


In [50]:
def limpiar_transacion(descripcion):
    
    common_words = [
        "DEP", "NOMINA", "CAJERO", "TRANSFER", "SPEI", "RECIBIDO", "PAGO", "TARJETA", 
        "VISA", "RETIRO", "AUTOMATICO", "CARGA", "ONLINE", "REF", "VENTAS", "PMTS", "AV",
        "MX", "SUC", "CDMX", "VTA", "DIRECTA", "INTERBANCARIA", "CLABE", "EN", "LINEA", 
        "COMPRA", "CHEQUE", "RENTA", "DEPTO", "STORE", "SERV", "CARGO", "AUTOMATICO", 
        "GYM", "REF", "REFORMA", "TAR", "P", "C", "A", "AUTOMATICA","COM","ZONA","ROSA","SERVICIO","MKTPLACE","TECHNOLOGY"
    ]
    cleaned = descripcion.upper()
    if "TIENDA3B" not in cleaned:
        cleaned = re.sub(r' \d+'," ", cleaned)
        cleaned = re.sub(r'[^A-Z\s]'," ", cleaned)
    if "TIENDA3B" in cleaned:
        cleaned = re.sub(" \d+", " ", cleaned)
    cleaned_description = cleaned.split()
    filtered_words = [word for word in cleaned_description if word not in common_words]
    cleared_description = ' '.join(filtered_words)
    return cleared_description


De acuerdo al seguimiento de la parte 2 , se opta por crear una función para limpiar datos, empezando con la limpieza para borrar datos a través de una lista de python, se crea una variable para que el texto de la columna descripción sea transformado a mayusculas, a partir de ahí se hace una condición para que la tienda3b mantenga su número y que las otras transacciones queden sin números o signos diferentes que le dan una lógica a la descripción de la transacción,seguido de ello , se declara una variable cleaned_description que nos servirá para guardar una lista de elementos cortados con el método split, lo anterior nos servira para pasar cada elemento de la variable cleaned_description en la lista de common words solo para iterar nuevamente los elementos que no estan en common words en la variable filtered words, por ultimo se une a través de la declarcion de una variable llamada cleared_description y se regresa el resultado.

In [8]:
test_limpiar_transaccion = 'TIENDA3B REFORMA 2453 COMPRA CHEQUE'
print(limpiar_transacion(test_limpiar_transaccion))

TIENDA3B


Antes de crear un update a la base de datos creamos un pequeño test para observar si funciona el método con un dato que debe de cumplir condiciones especiales.

In [9]:
transacciones = session.query(Transaccion).all()
for transaccion in transacciones:
    transaccion.cleaned_transaction = limpiar_transacion(transaccion.descripcion)
session.commit()


2024-07-24 07:39:01,631 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-24 07:39:01,633 INFO sqlalchemy.engine.Engine SELECT transacciones.id AS transacciones_id, transacciones.descripcion AS transacciones_descripcion, transacciones.cleaned_transaction AS transacciones_cleaned_transaction, transacciones.categoria AS transacciones_categoria, transacciones.logo AS transacciones_logo 
FROM transacciones
2024-07-24 07:39:01,634 INFO sqlalchemy.engine.Engine [generated in 0.00076s] ()
2024-07-24 07:39:01,637 INFO sqlalchemy.engine.Engine UPDATE transacciones SET cleaned_transaction=? WHERE transacciones.id = ?
2024-07-24 07:39:01,638 INFO sqlalchemy.engine.Engine [generated in 0.00070s] (('BBVA', 1), ('BBVA BANCOMER', 2), ('BANORTE', 3), ('AMAZON', 4), ('STRIPE UBEREATSMX', 5), ('HSBC', 6), ('SUBURBIA', 7), ('SPOTIFY SPOTIFY', 8)  ... displaying 10 of 20 total bound parameter sets ...  ('IZZI TELECOM', 19), ('ENERGY FITNESS', 20))
2024-07-24 07:39:01,641 INFO sqlalchemy.engine.Engine

Proseguimos a aplicar el método que creamos  en la columna cleaned_transaction de la base de datos y hacemos un update a la misma.

In [25]:
load_dotenv()
openai_api_key = os.getenv("API_KEY")
client = OpenAI(api_key=openai_api_key)
def categorizar_transaccion(cleaned_transaction):
    try:
        response = client.chat.completions.create(
            model = "gpt-3.5-turbo",
            messages=[
                {"role":"system","content":"Eres un asistente que categoriza en una palabra sin signos las transacciones"},
                {"role":"user","content":f"Categoriza esta transacción: {cleaned_transaction}"}
            ] 
        )
        category = response.choices[0].message.content.strip()
        return category
    except Exception as e:
        print(f'Error en la categoria: {e}')
        return 'Desconocido'

Para realizar las categorías en la base de datos utilizamos la API de opeanAI dentor de una función nombrada **categorizar_transaccion**  con el fin de tener mayor certeza en las categorías utilizamos el modelo gpt-3.5-turbo para conservar los costos a par de la calidad de la información que nos brindará la API, para ello importamos nuestras credencias alojadas en un archivo .env para mantener la seguridad de las mismas,  generamos un rol al chatbot y un rol para el usuario , realizamos dos pruebas para obserrvar la información que nos dará, se crea un excepción en caso de que se generé un error.

In [11]:
test_cleaned_transaction = "STRIPE UBER EATSMX"
print(categorizar_transaccion(test_cleaned_transaction))

ChatCompletion(id='chatcmpl-9oWUoziWKEt8R33EzqWr7y6qbRqGO', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='UberEatsMX', role='assistant', function_call=None, tool_calls=None))], created=1721828342, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=4, prompt_tokens=45, total_tokens=49))
UberEatsMX
UberEatsMX


Generamos un pequeño test para lograr observar como se comporta el chatbot y en caso de ser contrario a lo que ocupamos ajustarlo.

In [12]:
transacciones = session.query(Transaccion).all()
for transaccion in transacciones:
    if transaccion.cleaned_transaction:
        transaccion.categoria = categorizar_transaccion(transaccion.cleaned_transaction)
session.commit()


2024-07-24 07:39:02,330 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-24 07:39:02,331 INFO sqlalchemy.engine.Engine SELECT transacciones.id AS transacciones_id, transacciones.descripcion AS transacciones_descripcion, transacciones.cleaned_transaction AS transacciones_cleaned_transaction, transacciones.categoria AS transacciones_categoria, transacciones.logo AS transacciones_logo 
FROM transacciones
2024-07-24 07:39:02,331 INFO sqlalchemy.engine.Engine [cached since 0.6991s ago] ()
ChatCompletion(id='chatcmpl-9oWUonOVqkOVn0jBHZipxPpIxRmHB', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='banco', role='assistant', function_call=None, tool_calls=None))], created=1721828342, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=2, prompt_tokens=38, total_tokens=40))
banco
ChatCompletion(id='chatcmpl-9oWUpQSQLwFB5eaqWowP07yvcWTc8', choices=[Choice

Se aplica la función a la columna categoria, incorporando los datos de la columna cleaned_transaction para realizar el update.


## Parte 3: Búsqueda y Almacenamiento de Logos



In [13]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")


se cargan las apis keys de las tools que vamos utilizar para el agent de langchain como lo es openai y tavily

In [37]:
def logo_searcher(company_name):
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)
    search = TavilySearchAPIWrapper()
    tavily_tool = TavilySearchResults(api_wrapper=search)
    agent_chain = initialize_agent(
        [tavily_tool],
        llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
    )
    
    question = f"Como buscador de logos.Usa el nombre de la empresa: {company_name} para buscar y devolver la URL del logo. El paso a seguir es devolver solo la URL del logo actualizada. Se espera que devuelvas la url de los logos utilizando wikimedia como principal recurso y seeklogo como recurso secundario.Solo devuelve la url actualizada.Como ultimo recurso y solo en caso de no encontrar el logo devuelve esta url https://cdn.brandfetch.io/filenotfound.com.uy/w/130/h/145/logo"
    context = "Devuelve solo la URL actualizada"
    try:
        result = agent_chain.run(input={"question":question,"context":context})
        if not result:
            raise ValueError("No se encontró información")
    except Exception as e:
        result = "https://cdn.brandfetch.io/filenotfound.com.uy/w/130/h/145/logo"

    return result



- Se crea el agente de langchain utilizando el modelo gpt 3.5 turbo de chat gpt y la tool de tavilysearch result para buscar en la red los logos por medio de una question y un context que permiten contextualizar más el agente para una respuesta que tenga mayor relación a lo que se espera que realice, sin embargo se crea una excepción en el caso que no exista o no se pueda recuperar el logo a fin de insertar un valor por default.

- Para ejecutar adecuadamente el agente en su interior se centra en crear un prompt manteniendo los principios de Role Task Requirements Instructions con una organización de Role Requirements Task Expectation, lo cual ayuda al modelo gpt-3.5-turbo a mantener una tarea organizada junto con un contexto pertinente para desempeñarse correctamente.


In [36]:
print(logo_searcher('TIEND3B'))



> Entering new AgentExecutor chain...
{
  "action": "Final Answer",
  "action_input": "https://commons.wikimedia.org/wiki/File:Tiend3b_logo.png"
}

> Finished chain.
{
  "action": "Final Answer",
  "action_input": "https://commons.wikimedia.org/wiki/File:Tiend3b_logo.png"
}


Proseguimos a crear un pequeño test para observar como se comporta la función.

In [38]:
transacciones = session.query(Transaccion).all()
for transaccion in transacciones:
    transaccion.logo = logo_searcher(transaccion.cleaned_transaction)
session.commit()

2024-07-24 08:24:12,710 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-24 08:24:12,711 INFO sqlalchemy.engine.Engine SELECT transacciones.id AS transacciones_id, transacciones.descripcion AS transacciones_descripcion, transacciones.cleaned_transaction AS transacciones_cleaned_transaction, transacciones.categoria AS transacciones_categoria, transacciones.logo AS transacciones_logo 
FROM transacciones
2024-07-24 08:24:12,712 INFO sqlalchemy.engine.Engine [cached since 2711s ago] ()


> Entering new AgentExecutor chain...
Thought: The user is requesting to search for the logo of the company "BBVA" and provide the updated URL. The primary source for the logo search is Wikimedia, with Seeklogo as a secondary resource. If the logo is not found, a fallback URL should be provided.
Action:
```
{
  "action": "tavily_search_results_json",
  "action_input": {"query": "BBVA logo site:wikimedia.org"}
}
```
Observation: [{'url': 'https://commons.wikimedia.org/wiki/Category:BBVA_logos', 'conte

Creamos un update a la base de datos en la columna logo para consumir la función.


## Parte 4: Documentación y Pruebas

Crearemos un test unificado , a fin de observar los datos de cada función y analizar su comportamiento

In [70]:

transaction_dict = {}

for x in transacciones:
    transaction_clean = limpiar_transacion(x)
    transaction_dict[transaction_clean] = {
        "Categoria": categorizar_transaccion(transaction_clean),
        "Logo": logo_searcher(transaction_clean)
    }

df = pd.DataFrame.from_dict(transaction_dict, orient='index')
df




> Entering new AgentExecutor chain...
Thought: The user is asking for a logo search for the company BBVA. The request specifies using Wikimedia as the primary resource and Seeklogo as the secondary resource. If the logo is not found, a fallback URL is provided. I need to search for the logo using the company name and return the updated URL.

Action:
```
{
  "action": "tavily_search_results_json",
  "action_input": {"query": "BBVA logo site:wikimedia.org"}
}
```
Observation: HTTPError('429 Client Error: Too Many Requests for url: https://api.tavily.com/search')
Thought:Thought: The search engine tool encountered an error due to too many requests. I need to adjust the search query to avoid triggering this error.

Action:
```
{
  "action": "tavily_search_results_json",
  "action_input": {"query": "BBVA logo site:wikimedia.org"}
}
```
Observation: HTTPError('429 Client Error: Too Many Requests for url: https://api.tavily.com/search')
Thought:I encountered an error due to too many request

,Categoria,Logo
BBVA,Banco,https://cdn.brandfetch.io/filenotfound.com.uy/...
BBVA BANCOMER,Banco,https://commons.wikimedia.org/w/index.php?sear...
BANORTE,Banco,Thought: The user is asking for a logo search ...
AMAZON,Compras,https://cdn.brandfetch.io/filenotfound.com.uy/...
STRIPE UBEREATSMX,Comida,https://cdn.brandfetch.io/filenotfound.com.uy/...
HSBC,Banco,https://cdn.brandfetch.io/filenotfound.com.uy/...
SUBURBIA,Ropa,https://cdn.brandfetch.io/filenotfound.com.uy/...
SPOTIFY SPOTIFY,Entretenimiento,https://cdn.brandfetch.io/filenotfound.com.uy/...
OXXO TIENDA,Compra,"{\n ""action"": ""Final Answer"",\n ""action_inpu..."
BANAMEX,Banco,https://cdn.brandfetch.io/filenotfound.com.uy/...


Procederemos a crear un diccionario que guarde los resultados para simular la base de datos.

## Conclusión 



### Documentos externos

- https://brandfetch.com/developers
- https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/
- https://ucddublin.pressbooks.pub/StudentResourcev1_od/chapter/the-structure-of-a-good-prompt/
- https://docs.tavily.com/docs/gpt-researcher/config 
- https://www.promptingguide.ai/models/chatgpt 
